# PCA - Human Activity Recognition

En éste ejercicio vamos a usar datos provenientes de acelerómetros, giróscopos y otros sensores de smartphones Samsung Galaxy S3. A partir de esos sensores, se crearon distintas features (aceleración, jerk, ángulos, etc.) cuyo objetivo es permitir predecir distintos tipos de activididades que se etiquetaron en la experimentación. Para más información sobre los datos, consultar la fuente [original](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones).

### Importamos las librerías

<style>
  table {margin-left: 0 !important;}
</style>

In [ ]:
%pip install numpy seaborn pandas tqdm sklearn

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook

%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

RANDOM_STATE = 17

### Cargamos el dataset

- Bajar zip del dataset de https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
- Crear un directorio data/HAR_Dataset en el directorio actual de este archivo
- Descomprimir la carpeta "train" que se encuentra en el archivo zip, en el directorio data/HAR_Dataset

In [ ]:
PATH_DATASET = "./data/HAR_Dataset/train"

In [ ]:
X_train = np.loadtxt(os.path.join(PATH_DATASET, "X_train.txt"))
y_train = np.loadtxt(os.path.join(PATH_DATASET, "y_train.txt")).astype(int)

### EDA Mínimo

In [ ]:
print("Estructura features del dataset: {}".format(X_train.shape))
print("Estructura de las etiquetas: {}".format(y_train.shape))

In [ ]:
# Exploramos un poco las features
df = pd.DataFrame(data=X_train)
df.describe()

In [ ]:
df.isna().sum().sum()

In [ ]:
# Cantidad única de clases
clases = np.unique(y_train)
clases

In [ ]:
n_clases = clases.size

In [ ]:
n_clases

|     Label     |   Descripción  | 
| ------------- |:-------------: |
|       1       | Caminar        | 
|       2       | Subir escaleras|
|       3       | Bajar escaleras|
|       4       | Estar sentado  |
|       5       | Estar parado   |
|       6       | Recostarse     |

### Aplicamos PCA

In [ ]:
# Estandarizamos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

Al aplicar PCA, es una práctica común reducir el número de dimensiones, dejando tantos componentes como sean necesarios para que contemplen al menos el 90% de la varianza de los datos escalados originales. Scikit-learn permite indicar directamente ese parámetro como condición. Si no se puede seleccionar un número alto de componentes y mediante un scree plot, verificar cuántos son necesarios para mantener un 90%.

In [ ]:
pca = PCA(n_components=0.9, random_state=RANDOM_STATE).fit(X_scaled)
X_pca = pca.transform(X_scaled)

In [ ]:
X_pca.shape

In [ ]:
# Cantidad de componentes necesarios
X_pca.shape[1]

In [ ]:
pca_2 = PCA(n_components=200, random_state=RANDOM_STATE).fit(X_scaled)

plt.figure(figsize=(10,7))
plt.plot(np.cumsum(pca_2.explained_variance_ratio_), color='k', lw=2)
plt.xlabel('Cantidad de componentes')
plt.ylabel('Total varianza contemplada')
plt.xlim(0, 200)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.axvline(63, c='b')
plt.axhline(0.9, c='r')
plt.show();

In [ ]:
# Graficamos los primeros dos componentes
plt.figure(figsize=(12,10))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_train, alpha=0.7, s=40, cmap=plt.cm.get_cmap('nipy_spectral', 6))
plt.rcParams['axes.grid'] = False #Evita un warning
plt.colorbar()
plt.title('HAR - PCA projection 2D');

### Clusterización

![image](https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_001.png)

In [ ]:
kmeans = KMeans(n_clusters=6, n_init=100, random_state=RANDOM_STATE)
kmeans.fit(X_pca)
cluster_labels = kmeans.labels_

In [ ]:
# Graficamos los primeros dos componentes - clusters id
plt.figure(figsize=(12,10))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, edgecolor='none', alpha=0.7, s=40, cmap=plt.cm.get_cmap('nipy_spectral', 6))
plt.rcParams['axes.grid'] = False
plt.colorbar()
plt.title('HAR - PCA projection 2D');

In [ ]:
kmeans_move = KMeans(n_clusters=2, n_init=100, random_state=RANDOM_STATE)
kmeans_move.fit(X_pca)
cluster_labels_mov = kmeans_move.labels_

In [ ]:
# Graficamos los primeros dos componentes - clusters id
plt.figure(figsize=(12,10))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels_mov, edgecolor='none', alpha=0.7, s=40, cmap=plt.cm.get_cmap('nipy_spectral', 2))
plt.colorbar()
plt.title('HAR - PCA projection 2D');

In [ ]:
tab = pd.crosstab(y_train, cluster_labels, margins=True)
tab.index = ['Caminar', 'Subir escaleras', 'Bajar escaleras', 'Estar parado', 'Estar sentado', 'Recostarse', 'Todos']
tab.columns = ['cluster ' + str(i + 0) for i in range(6)] + ['Todos']
tab

In [ ]:
tab = pd.crosstab(y_train, cluster_labels_mov, margins=True)
tab.index = ['Caminar', 'Subir escaleras', 'Bajar escaleras', 'Estar parado', 'Estar sentado', 'Recostarse', 'Todos']
tab.columns = ['cluster ' + str(i + 0) for i in range(2)] + ['Todos']
tab

### K-means - Inercia

In [ ]:
# Vemos la inercia (suma de las distancias de los miembros de un cluster a su centroide)
inertia = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, n_init=100, random_state=RANDOM_STATE).fit(X_pca)
    inertia.append(np.sqrt(kmeans.inertia_))

$$\Large J(C) = \sum_{k=1}^K\sum_{i~\in~C_k} ||x_i - \mu_k|| \rightarrow \min\limits_C,$$

$C$ – es un set de clusters de orden $K$, $\mu_k$ es el centroide del cluster $C_k$.

In [ ]:
plt.plot(range(1, 10), inertia, marker='s')
plt.show()

Se quiere minimizar la Inercia, pero el óptimo está en K = N, por lo tanto se introduce la variable D, que mide las proporciones de diferencia de inercia entre K sucesivos. 

$$\Large D(k) = \frac{|J(C_k) - J(C_{k+1})|}{|J(C_{k-1}) - J(C_k)|}  \rightarrow \min\limits_k $$

In [ ]:
d = {}
for k in range(2, 9):
    i = k - 1
    d[k] = (inertia[i] - inertia[i + 1])  / (inertia[i - 1] - inertia[i]) 

In [ ]:
d

---

### Bibliografía

* [Performance Metrics in Machine Learning — Part 3: Clustering](https://towardsdatascience.com/performance-metrics-in-machine-learning-part-3-clustering-d69550662dc6)

* [Kaggle Notebook - 1](https://www.kaggle.com/kashnitsky/a7-demo-unsupervised-learning-solution?select=samsung_test_labels.txt)

* [Información sobre el dataset](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones)